In [ ]:
import os
import pickle
import re
from datetime import datetime
from os import path
from typing import Dict

from commonroad.common.util import Interval
from commonroad.scenario.scenario import Tag

from BatchConversion.BatchConverter import BatchConverter, BatchConversionResult
from OpenSCENARIO2CR.ConversionAnalyzer.EAnalyzer import EAnalyzer
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.EsminiWrapperProvider import EsminiWrapperProvider
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.StoryBoardElement import EStoryBoardElementType
from OpenSCENARIO2CR.Osc2CrConverter import Osc2CrConverter
from OpenSCENARIO2CR.util.AbsRel import AbsRel
from OpenSCENARIO2CR.util.PlanningProblemBuilder import PlanningProblemSetBuilder
from OpenSCENARIO2CR.util.Serializable import Serializable


In [ ]:
esmini_wrapper = EsminiWrapperProvider().provide_esmini_wrapper()
esmini_wrapper.max_time = 120.0
esmini_wrapper.grace_time = 1.0
esmini_wrapper.ignored_level = EStoryBoardElementType.ACT
esmini_wrapper.log_to_console = False
esmini_wrapper.log_to_file = False
esmini_wrapper.random_seed = 0

In [ ]:
pps_builder = PlanningProblemSetBuilder()
pps_builder.goal_state_time_step = AbsRel(Interval(-10, 0), False)
pps_builder.goal_state_position_length = 50
pps_builder.goal_state_position_width = 10
pps_builder.goal_state_position_use_ego_rotation = True
pps_builder.goal_state_orientation = None
pps_builder.goal_state_velocity = None
# If you want to also include orientation and velocity in the GoalRegion provide some AbsRel objects
# pps_builder.goal_state_orientation = AbsRel(AngleInterval(-0.8, 0.8), is_absolute=False)
# pps_builder.goal_state_velocity = AbsRel(Interval(-10, 10), is_absolute=False)

In [ ]:


converter = Osc2CrConverter(
    author="ADD AUTHOR HERE",
    affiliation="ADD AFFILIATION HERE",
    source="ADD SOURCE HERE",
    tags={Tag.SIMULATED},
)

converter.sim_wrapper = esmini_wrapper
converter.pps_builder = pps_builder

converter.dt_cr = 0.1
converter.keep_ego_vehicle = True
converter.trim_scenario = False
converter.use_implicit_odr_file = True
converter.analyzers = [
    EAnalyzer.SPOT_ANALYZER,
    EAnalyzer.DRIVABILITY_CHECKER,  # Will be initialized with default parameters
    EAnalyzer.STL_MONITOR,  # Will be initialized with default parameters
]
# If you don't want to run with default parameters for (some) analyzers use:
# converter.analyzers = {
#     EAnalyzer.SPOT_ANALYZER: SpotAnalyzer(timeout=30),
#     EAnalyzer.DRIVABILITY_CHECKER: None,  # Will be initialized with default parameters
#     EAnalyzer.STL_MONITOR: None,  # Will be initialized with default parameters
# }

converter.dt_sim = 0.01
converter.odr_file_override = None
converter.ego_filter = re.compile(r".*ego.*", re.IGNORECASE)


In [ ]:
# Configured see the previous cells
batch_converter = BatchConverter(converter)
# If the default parameters are sufficient the following line is sufficient
# batch_converter = BatchConverter(Osc2CrConverter())

In [ ]:
from os.path import expanduser

batch_converter.discover_files(expanduser("~"), re.compile(r".*\.xosc", re.IGNORECASE), recursively=True)

In [ ]:
time_conversion_started = datetime.now()
results: Dict[str, BatchConversionResult] = batch_converter.run_batch_conversion(num_worker=0)
time_conversion_finished = datetime.now()

In [ ]:
storage_path = "results/{}".format(datetime.now().isoformat(sep="_", timespec="seconds"))
store = True
store_scenario_files = True

if store:
    os.makedirs(storage_path, exist_ok=True)
    with open(os.path.join(storage_path, "statistics.pickle"), "wb") as file:
        Serializable.storage_dir = storage_path
        Serializable.store_extra_files = store_scenario_files
        pickle.dump(results, file)
    print(f"Finished and stored results in {path.abspath(storage_path)}")


In [ ]:
from OpenSCENARIO2CR.ConversionAnalyzer.AnalyzerErrorResult import AnalyzerErrorResult

errors: Dict[AnalyzerErrorResult, int] = {}
for result in results.values():
    if not result.without_exception:
        if result.exception in errors:
            errors[result.exception] += 1
        else:
            errors[result.exception] = 1

for error, count in errors.items():
    print(f"{count}\n{error.traceback_text}")
    print("\n" * 3)